![image](dependencies/open-science-prize.png)

## Higher Fidelity Graph States
In this notebook, we will prepare quantum circuits for a 7-qubit graph state and estimate the fidelity through stabilizer measurements using Qiskit. In this example, we use the CTMP method of error mitigation [1] and repeat the graph state measurement 16 times to find statistical error bars.

**To Do:
Modify the graph state preparation circuit or use your own methods of error mitigation to improve the graph state fidelity.**


[1] S. Bravyi, S. Sheldon, A. Kandala, D.C. McKay, J.M. Gambetta, Mitigating measurement errors in multi-qubit experiments, [arXiv:2006.14044](https://arxiv.org/abs/2006.14044) (2020).

## Imports

Begin by importing the necessary packages and defining the functions we will need for the stabilizer measurements.

In [ ]:
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
import qiskit.ignis.mitigation as mit

# Qiskit tools for noisy simulation
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.utils import insert_noise

# Qiskit tools for running and monitoring jobs
from qiskit import execute
from qiskit.tools.monitor import job_monitor

# Other imports
import numpy as np

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In order to run the circuits, first load the backend `ibmq_casablanca` from your account using the `IBMQ` provider. You will receive access to `ibm-q-community/ibmquantumawards/open-science` after registering for the Open Science Prize.

In [ ]:
# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science')
backend = provider.get_backend('ibmq_casablanca')
properties = backend.properties()

## Preparing graph states

Here, we prepare the graph state circuit for 7-qubits using the `GraphState` function in Qiskit's circuit library. We define a graph that uses the connectivity map of the quantum system `ibmq_casablanca`.

In [ ]:
num_qubits = 7

# adjacency matrix for `ibmq_casablanca`
adjmat = [
    [0, 1, 0, 0, 0, 0, 0], 
    [1, 0, 1, 1, 0, 0, 0], 
    [0, 1, 0, 0, 0, 0, 0], 
    [0, 1, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 1, 1, 0, 1], 
    [0, 0, 0, 0, 0, 1, 0]]

In [ ]:
from osp.cross_talk import to_cross_talk_impact, benchmarks
from osp.plot import plot_cross_talk_impact
cross_talk_impacts = to_cross_talk_impact(benchmarks[0]["single_cx_errors"], benchmarks[0]["cross_talk_errors"])
plot_cross_talk_impact(cross_talk_impacts)

In [ ]:
cross_talk_impacts = to_cross_talk_impact(benchmarks[1]["single_cx_errors"], benchmarks[1]["cross_talk_errors"])
plot_cross_talk_impact(cross_talk_impacts)

### Your code goes here

How would you prepare a graph state with high fidelity? In the example below, we create it using Qiskit's circuit library at the gate level. You may explore other methods for creating the graph states, including by using pulse-level techniques or accounting for the errors in the quantum system.

In [ ]:
from osp.graph_state import GraphState

def create_graph_state(backend, cross_talk_properties = None, cross_talk_threshold = 1.5):
    
    ### YOUR CODE GOES HERE -- START

    graph_state_circuit = GraphState(backend, cross_talk_properties, cross_talk_impact_threshold=cross_talk_threshold)
    
    ### YOUR CODE GOES HERE -- END
    
    return graph_state_circuit

In [ ]:
# the graph state can be created using Qiskit's circuit library
state_circuit = create_graph_state(backend, cross_talk_impacts)
state_circuit.draw(output="mpl")

In [ ]:
from qiskit import transpile
compiled_state_circuit = transpile(state_circuit, backend=backend)
compiled_state_circuit.draw(output="mpl")

In [ ]:
from qiskit import schedule
from qiskit.visualization import pulse_drawer
schd = schedule(compiled_state_circuit, backend)
pulse_drawer(schd)

## Qiskit version

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table